In [398]:
y_wave = pow(h, x2, p)
y_wave

99524571567389865764528099515781141894963043343204968711202652520115748140992492450738364758361395345408976732416572028187157981105564799136557094376569103818735421899422547291243296376836355178733023813258919739668602683407740608851146552396298993783579490229819823473668450018454515256603737074398733001609

In [399]:
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.backends import default_backend
import random
from hashlib import sha256, sha3_256


def generate_dsa_parameters():
    # Generate DSA parameters with 1024-bit p and 160-bit q
    parameters = dsa.generate_parameters(key_size=1024, backend=default_backend())
    
    # Generate private and public keys based on the generated parameters
    private_key = parameters.generate_private_key()
    public_key = private_key.public_key()

    # Access the DSA parameters
    p = parameters.parameter_numbers().p
    q = parameters.parameter_numbers().q
    g = parameters.parameter_numbers().g

    print("Generated DSA Parameters:")
    print("p:", p)
    print("q:", q)
    print("g:", g)
    print("\nPrivate Key:", private_key)
    print("Public Key:", public_key)
    return p,q,g

p,q,g = generate_dsa_parameters()

def generate_keys(g, p):
    # 生成私钥s
    s = random.randint(1, p-2)
    # 计算公钥v = g^s mod p
    v = pow(g, s, p)
    return s, v

msg = 'Hello, Ring Signature'
print('msg:', msg)

x1, y1 = generate_keys(g, p)
x2, y2 = generate_keys(g, p)
x3, y3 = generate_keys(g, p)
public_key_list = [y1, y2, y3]
print('pubkeys:', public_key_list)
Y = ''.join([str(i) for i in public_key_list])
print('Y:', Y)

h = int(sha3_256(str(Y).encode()).hexdigest(), 16) % p
h = pow(g, h, p)
y_wave = pow(h, x2, p)

r, R = generate_keys(g, q)
e3 = int(sha256(str(str(msg) + str(Y) + str(pow(g, r, p)) + str(y_wave) + str(pow(h, r, p))).encode()).hexdigest(), 16) % q
print('e3:', e3)

_, random_s3 = generate_keys(g, q)
print('random_s3:', random_s3)

calculation = (pow(g, random_s3, p)*pow(y3, e3, p)) % p
calculation_1 = (pow(h, random_s3, p)*pow(y_wave, e3, p)) % p
e1 = int(sha256(str(str(msg) + str(Y) + str(calculation) + str(y_wave) + str(calculation_1)).encode()).hexdigest(), 16) % q
print('e1:', e1)

_, random_s1 = generate_keys(g, q)
print('random_s1:', random_s1)

calculation = (pow(g, random_s1, p)*pow(y1, e1, p)) % p
calculation_1 = (pow(h, random_s1, p)*pow(y_wave, e1, p)) % p
e2 = int(sha256((str(msg) + str(Y) + str(calculation) + str(y_wave) + str(calculation_1)).encode()).hexdigest(), 16) % q
print('e2:', e2)

s2 = (r - x2 * e2) % ((p-1)//2)
print('s2:', s2)

total_res = {
    'e1': e1,
    's1': random_s1,
    's2': s2,
    's3': random_s3
}

print('signature process total result:')
display(total_res)

Generated DSA Parameters:
p: 130046647850388264320143775188060697096610325119132783454283941651796270228178494460573865919971478739543325143613180783495007439135955899680624535978966946245668959414504184249488219631777404897569869807067600501622037064344343657872849212088587728699871494283912782073776881875755553005562891650138478842293
q: 1300980027076646086143239961901213243924483914227
g: 103243008824125050700390254697040674192013424299948706857347960309690276676655948201384822575488133788872939315996667501097433520245010830741490836009678707893339515839381849874568711953528393298586215139039097923765732154634836079545480503371127419734639134273360522244190886483967403936001189639977730354312

Private Key: <cryptography.hazmat.bindings._rust.openssl.dsa.DSAPrivateKey object at 0x00000276C540B750>
Public Key: <cryptography.hazmat.bindings._rust.openssl.dsa.DSAPublicKey object at 0x00000276C54090D0>
msg: Hello, Ring Signature
pubkeys: [4940936298367609339762838946917656898276187134997

{'e1': 210378308148480253079172626400845198477300885491,
 's1': 52963575060521116058391439155557554803446486396,
 's2': 26773897151534343152375885153782982730703475260112096466135125796185744047040912611067272614003814324986209326032667375603905503971696227429752888489522216738481583660167002649640650081227240915131095782371730363067838038757197135886096141358374832213702128922311181185078577453988985972391280963212972765474,
 's3': 192837513579391262307705832911433945825445633940}

In [400]:
calculation = (pow(g, total_res['s1'], p)*pow(y1, total_res['e1'], p)) % p
calculation_1 = (pow(h, total_res['s1'], p)*pow(y_wave, total_res['e1'], p)) % p
e2_V = int(sha256((str(msg) + str(Y) + str(calculation) + str(y_wave) + str(calculation_1)).encode()).hexdigest(), 16) % q
calculation = (pow(g, total_res['s2'], p) * pow(y2, e2_V, p)) % p
calculation_1 = (pow(h, total_res['s2'], p)*pow(y_wave, e2_V, p)) % p
e3_V = int(sha256((str(msg) + str(Y) + str(calculation) + str(y_wave) + str(calculation_1)).encode()).hexdigest(), 16) % q
calculation = (pow(g, total_res['s3'], p)*pow(y3, e3_V, p)) % p
calculation_1 = (pow(h, total_res['s3'], p)*pow(y_wave, e3_V, p)) % p
e1_V = int(sha256((str(msg) + str(Y) + str(calculation) + str(y_wave) + str(calculation_1)).encode()).hexdigest(), 16) % q

print(e1_V)
print(e2_V)
print(e3_V)
print(e1_V==e1)

210378308148480253079172626400845198477300885491
774001091531025821808832396543798643506898872622
61337237892963278602028347173905347343219196676
True
